In [49]:
import pandas as pd
import numpy as np
import os
from pyreadr import read_r

In [50]:
dir_list = [
    'bagru1',
    'bagru2',
    'jobner',
    'dand',
    'sawarda'
]

In [51]:
month_list = [
    'oct_fuel.csv',
    'nov_fuel.csv',
    'dec_fuel.csv',
    'jan_fuel.csv',
    'feb_fuel.csv',
    'feb2_fuel.csv'
]

In [52]:
df_list = []

In [53]:
for directory in dir_list:
    for file in month_list:
        df = pd.read_csv(f"{directory}/{file}")
        df_list.append(df)

In [54]:
len(df_list)

30

In [55]:
out = pd.concat(df_list)

In [56]:
out

,site,ts,rV,aV
0,Bagru1,2021-09-30T18:30:09Z,54.2,NaN
1,Bagru1,2021-09-30T18:30:39Z,54.2,NaN
2,Bagru1,2021-09-30T18:31:09Z,54.2,NaN
3,Bagru1,2021-09-30T18:31:39Z,54.2,NaN
4,Bagru1,2021-09-30T18:32:09Z,54.2,NaN
...,...,...,...,...
3964,Sawarda,2022-02-28T18:28:13Z,28.9,NaN
3965,Sawarda,2022-02-28T18:28:43Z,28.9,NaN
3966,Sawarda,2022-02-28T18:29:13Z,28.9,NaN
3967,Sawarda,2022-02-28T18:29:13Z,28.9,NaN


In [57]:
from datetime import date, timedelta

date_list = []
start = date(2021, 10, 1)
end = date(2022, 2, 28)

d = start
while d<=end:
    date_list.append(d)
    d+=timedelta(days=1)

In [58]:
print(date_list[0], date_list[-1])

2021-10-01 2022-02-28


In [59]:
out.ts = pd.to_datetime(out.ts)

In [60]:
import pytz

IST = pytz.timezone("Asia/Kolkata")

In [61]:
out.ts = out.ts.apply(lambda x: x.astimezone(IST))

In [62]:
out

,site,ts,rV,aV
0,Bagru1,2021-10-01 00:00:09+05:30,54.2,NaN
1,Bagru1,2021-10-01 00:00:39+05:30,54.2,NaN
2,Bagru1,2021-10-01 00:01:09+05:30,54.2,NaN
3,Bagru1,2021-10-01 00:01:39+05:30,54.2,NaN
4,Bagru1,2021-10-01 00:02:09+05:30,54.2,NaN
...,...,...,...,...
3964,Sawarda,2022-02-28 23:58:13+05:30,28.9,NaN
3965,Sawarda,2022-02-28 23:58:43+05:30,28.9,NaN
3966,Sawarda,2022-02-28 23:59:13+05:30,28.9,NaN
3967,Sawarda,2022-02-28 23:59:13+05:30,28.9,NaN


In [63]:
out['date'] = out.ts.dt.date.values

In [64]:
from tqdm import tqdm
from datetime import datetime, time

missing_hrs_list = []
site_list = []

de = pd.Timedelta("5 minutes")
print(de)

for site in dir_list:
    for d in tqdm(date_list):
        time_stamps = []
        start_time = IST.localize(datetime.combine(d, time(0, 0, 0)))
        end_time = IST.localize(datetime.combine(d, time(23, 59, 59)))
        
        missing_hrs = 0
        
        tmp = out[(out['site']==site.title()) & (out['date']==d)].reset_index(drop=True)
        for i, row in tmp.iterrows():
            # checking for initial boundary data miss
            if i == 0:
              cons_diff = abs(tmp.loc[i, f'ts'] - start_time)
                
              if cons_diff > de:
                time_stamps.append((start_time, row[f'ts']))
                missing_hrs += cons_diff.total_seconds()/3600
            # checking for intermediate data miss
            elif i < (len(tmp)-1):
              cons_diff = abs(tmp.loc[i+1, f'ts'] - tmp.loc[i, f'ts'])
              if cons_diff > de:
                time_stamps.append((row[f'ts'], tmp.loc[i+1, f'ts']))
                missing_hrs += cons_diff.total_seconds()/3600
            # checking for final boundary data miss
            else:
              cons_diff = abs(end_time - tmp.loc[i, f'ts'])
              if cons_diff > de:
                time_stamps.append((row[f'ts'], end_time))
                missing_hrs += cons_diff.total_seconds()/3600
            
        missing_hrs_list.append(missing_hrs)
        site_list.append(site.title())

0 days 00:05:00


100%|████████████████████████████████████████████████████████████████████████████████| 151/151 [01:13<00:00,  2.07it/s]


In [65]:
len(site_list)

755

In [66]:
res_df = pd.DataFrame({
    'site': site_list,
    'date': date_list*5,
    'missing_hrs': missing_hrs_list
})

In [67]:
res_df

,site,date,missing_hrs
0,Bagru1,2021-10-01,0.0
1,Bagru1,2021-10-02,0.0
2,Bagru1,2021-10-03,0.0
3,Bagru1,2021-10-04,0.0
4,Bagru1,2021-10-05,0.0
...,...,...,...
750,Sawarda,2022-02-24,0.0
751,Sawarda,2022-02-25,0.0
752,Sawarda,2022-02-26,0.0
753,Sawarda,2022-02-27,0.0


In [68]:
res_df.to_csv("missing_fuel_data.csv", index=False)